In [1]:
%%capture
# %% [markdown]
# Extended Audio Watermarking Analysis
# University of Zimbabwe - Computer Engineering Project 
# by Mayibongwe Moyo R214568M  
# Supervisor: Mr P Worsnop

# %% [markdown]
# ## 1. Baseline Watermarking (Original Example)
# Reproduce the original AudioSeal example with enhanced documentation

import sys
# Install dependencies with version locking
!{sys.executable} -m pip install torchaudio soundfile matplotlib scipy
import torch
import torchaudio
import urllib

In [2]:
# %% [code]
def download_sample_audio():
    """Download LJ-Speech sample with integrity check"""
    url = "https://keithito.com/LJ-Speech-Dataset/LJ037-0171.wav"
    with open("test.wav", "wb") as f:
        resp = urllib.request.urlopen(url)
        f.write(resp.read())
    
    # Verify audio properties
    wav, sample_rate = torchaudio.load("test.wav")
    assert sample_rate == 22050, "Unexpected sample rate"
    assert wav.shape[1] > 16000, "Audio too short"
    return wav, sample_rate


In [3]:
# %% [code]
# Data preparation with academic formatting
audio, sr = download_sample_audio()
print(f"Original Audio Properties:\n- Duration: {audio.shape[1]/sr:.2f}s\n- Channels: {audio.shape[0]}\n- Sample Rate: {sr}Hz")


Original Audio Properties:
- Duration: 7.58s
- Channels: 1
- Sample Rate: 22050Hz


In [4]:
# %% [code]
# Enhanced visualization function
def plot_analysis(waveform, sr, title):
    """Publication-quality visualization"""
    plt.figure(figsize=(20, 5))
    
    # Waveform plot
    plt.subplot(1, 2, 1)
    plt.plot(waveform.numpy().T)
    plt.title(f"{title}\nTime Domain", fontsize=14)
    plt.xlabel("Sample Index", fontsize=12)
    plt.ylabel("Amplitude", fontsize=12)
    
    # Spectrogram plot
    plt.subplot(1, 2, 2)
    plt.specgram(waveform.numpy().flatten(), Fs=sr, NFFT=2048)
    plt.title(f"{title}\nFrequency Domain", fontsize=14)
    plt.xlabel("Time (s)", fontsize=12)
    plt.ylabel("Frequency (Hz)", fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [5]:
# %% [code]
# Modified watermarking loop with research metrics
def incremental_watermark(audio, sr, num_watermarks=3, alpha=0.15):
    """Sequential watermark embedding with analysis"""
    from IPython.display import clear_output
    from audioseal import AudioSeal
    
    # Initialize models and storage
    generator = AudioSeal.load_generator("audioseal_wm_16bits")
    detector = AudioSeal.load_detector("audioseal_detector_16bits")
    
    watermarked = audio.clone()
    results = []
    
    for i in range(num_watermarks):
        # 1. Generate unique message
        msg = torch.randint(0, 2, (1, 16))
        
        # 2. Embed watermark
        watermark = generator.get_watermark(
            watermarked.unsqueeze(0),
            sample_rate=sr,
            message=msg
        )
        watermarked += alpha * watermark.squeeze()
        
        # 3. Calculate metrics
        noise = watermarked - audio
        snr = 10 * torch.log10(audio.pow(2).mean() / noise.pow(2).mean())
        
        # 4. Detect all watermarks
        detector_results = []
        for m in range(i+1):
            detector.message = results[m]['message']
            prob, decoded = detector.detect_watermark(
                watermarked.unsqueeze(0),
                sample_rate=sr
            )
            ber = (results[m]['message'] != decoded.round()).float().mean()
            detector_results.append({
                'watermark': m+1,
                'ber': ber.item(),
                'prob': prob
            })
        
        # 5. Store results
        results.append({
            'iteration': i+1,
            'message': msg,
            'snr': snr.item(),
            'detections': detector_results
        })
        
        # 6. Dynamic visualization
        clear_output(wait=True)
        print(f"After {i+1} watermarks:")
        plot_analysis(watermarked, sr, f"Iteration {i+1}")
        display(Audio(watermarked.numpy(), rate=sr))
        
        # 7. Tabular results
        print(f"\n{' Watermark ':=^40}")
        print(f"| {'ID':<5} | {'BER':<8} | {'Detection Prob':<14} |")
        for res in detector_results:
            print(f"| {res['watermark']:>3} | {res['ber']:>6.2f} | {res['prob']:>12.2f} |")
        print(f"\nSNR: {snr:.2f} dB\n")
    
    return watermarked, results


In [6]:

# %% [code]
# Execute analysis with research parameters
final_audio, metrics = incremental_watermark(
    audio.squeeze(0),  # Remove channel dimension for processing
    sr=22050,
    num_watermarks=3,
    alpha=0.18
)

ValueError: not enough values to unpack (expected 3, got 2)